# Visualising PAG neurons in CCF space
In this notebook we will load the .csv file containing the metadata from our PAG_scRNAseq project and use the CCF coordinates obtained after registration with Sharp-Track to visualise our sequenced cells with Brainrender.

### 1 | Import packages and set defaults

In [2]:
import brainrender
from brainrender.scene import Scene, DualScene, MultiScene
from brainrender.colors import makePalette, get_random_colors
from vedo import embedWindow
embedWindow(False)  # Disables scene embedding in jupyter notebooks
from brainrender.animation.video import BasicVideoMaker as VideoMaker
import pandas as pd # used to load the cwv

# // BRAINRENDER DEFAULT SETTINGS //
# Change some of the default settings
brainrender.SHADER_STYLE = "cartoon" # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.BACKGROUND_COLOR = "white" # color of the background window (defaults to "white", try "blackboard")
brainrender.ROOT_COLOR = [0.4, 0.4, 0.4] # color of the overall brain model's actor (defaults to [0.4, 0.4, 0.4])
brainrender.ROOT_ALPHA = 0.2 # transparency of the overall brain model's actor (defaults to 0.2)
brainrender.DEFAULT_SCREENSHOT_NAME = "PAG_aspirated_cells" # screenshots will have this name and the time at which they were taken
brainrender.DEFAULT_SCREENSHOT_TYPE = ".png" # png, svg or jpg supported
brainrender.DEFAULT_SCREENSHOT_SCALE = 1 # values >1 yield higher resolution screenshots
brainrender.SCREENSHOT_TRANSPARENT_BACKGROUND = True # whether to save screenshots with transparent background

# // SET PARAMETERS //
# Save folder
save_folder = "D:/Dropbox (UCL - SWC)/Project_transcriptomics/analysis/PAG_scRNAseq_brainrender/output"

# Create screenshot parameters
screenshot_params = dict(folder = save_folder, name = "PAG_scRNAseq_brainrender")

#### 1.1 | Check atlas availability
Brainrender integrates several atlases that can be used to visualise and explore brain anatomy. We can check which atlases are available, take a look at the ones we have already downloaded, and render a basic scene with the axis to get an idea of the overall picture.

In [3]:
# Run this to get a list of the available atlases:
import brainatlas_api
from brainatlas_api import list_atlases
list_atlases()

Currently, we can use the following atlases for mouse data:
* allen_mouse_25um_v0.2 - Volume dimension of \[AP, SI, LR\] equivalent to \[13200, 8000, 11400\] (default, although it is really at 1um resolution) 
* kim_unified_25um_v0.1 - Volume dimension of \[AP, SI, LR\] equivalent to \[528, 320, 456\]
* kim_unified_50um_v0.1 - Volume dimension of \[AP, SI, LR\] equivalent to \[264, 160, 228\]

The CCF coordinates we obtained from Sharp-Track are at 10um resolution, with a Volume dimension of \[AP, SI, LR\] equivalent to \[1320, 800, 1140\]

In [17]:
# Create a scene with the with the preferred atlas and check the dimensions
brainrender.SHOW_AXES = True
scene = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params
scene.render() # make sure to press 'Esc' to close not 'q' or kernel diesscene.render(interactive = True, video = False, camera = "sagittal", zoom = 1)



Rendering scene.
   Press 'q' to Quit


### 2 | Load metadata including CCF coordinates
Other options to load registered points include `add_cells_from_file` and `get_probe_points_from_sharptrack`.

In [5]:
# Load data
pag_data = pd.read_csv("D:\\Dropbox (UCL - SWC)\\Project_transcriptomics\\analysis\\PAG_scRNAseq_brainrender\\PAG_scRNAseq_metadata_200617.csv")

# Look at the first 5 rows of the metadata
pag_data.head()

,Unnamed: 0,sequencing.id,date.collection,sample.id,sample.number,mouse.id,mouse.sex,mouse.age,mouse.singlehousedays,cell.id,...,RT.machine,RT.time,preamp.date,preamp.machine,preamp.time,date.purification,date.nanodrop,date.qubit,date.bioanalyzer,batch.sequencing_round
0,1,GC.OP.7836.EBR,180518,dmpag_vglut2_180518_c20_EBR,20,CE51.2,F,7.0,0,EBR,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
1,2,GC.OP.7836.EBT,180518,dmpag_vglut2_180518_c22_EBT,22,CE51.2,F,7.0,0,EBT,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
2,3,GC.OP.7836.EBU,180518,dlpag_vglut2_180518_c23_EBU,23,CE51.2,F,7.0,0,EBU,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
3,4,GC.OP.7836.EBV,180518,lpag_vglut2_180518_c24_EBV,24,CE51.2,F,7.0,0,EBV,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
4,5,GC.OP.7836.ECZ,180607,dlpag_vglut2_180607_c6_ECZ,6,BJ162.2,M,7.3,0,ECZ,...,classic,22:00,180608,non-classic,14:17,180609,180609.0,180609.0,180609.0,1


In [6]:
# List all column names in data
pag_data.columns

Index(['Unnamed: 0', 'sequencing.id', 'date.collection', 'sample.id',
       'sample.number', 'mouse.id', 'mouse.sex', 'mouse.age',
       'mouse.singlehousedays', 'cell.id', 'cell.type', 'cell.fluorophore',
       'patch.recorded', 'pipette.RNAseOUT', 'slice.number', 'slice.depth',
       'CCF.AllenAP', 'CCF.AllenDV', 'CCF.AllenML', 'CCF.AllenIndex',
       'CCF.AllenArea', 'CCF.PaxinosIndex', 'CCF.PaxinosArea',
       'PAG.hemisphere', 'PAG.PaxinosAPmanual', 'PAG.areacollection',
       'PAG.areamanualregistration', 'PAG.arearegistration', 'time.aspiration',
       'time.slicing', 'time.sinceslicingexact', 'time.sinceslicinghour',
       'batch.processing', 'RT.oligo', 'RT.date', 'RT.machine', 'RT.time',
       'preamp.date', 'preamp.machine', 'preamp.time', 'date.purification',
       'date.nanodrop', 'date.qubit', 'date.bioanalyzer',
       'batch.sequencing_round'],
      dtype='object')

#### 2.1 | Scaling up coordinates
The CCF coordinates were obtained by registering images to the Allen Brain Atlas using Sharp-Track (see Shamash et al. bioRxiv 2018 and https://github.com/cortex-lab/allenCCF), which yields coordinates at a resolution of 10 micrometers. In the Allen Brain Atlas, a point at coordinates \[1, 0, 0\] is at 10um from the origin (in other words, 1 unit of the atlas space equals 10um). However, BrainRender's space is at 1um resolution, so the first thing we need to do is to scale up the coordinate values by 10 to get them to match correctly.

In [7]:
# Inspect the CCF coordinates for each cell
pag_data[["cell.id", "cell.type", "PAG.areamanualregistration", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

,cell.id,cell.type,PAG.areamanualregistration,CCF.AllenAP,CCF.AllenDV,CCF.AllenML
0,EBR,VGluT2,dmpag,993,235,549
1,EBT,VGluT2,dmpag,993,231,594
2,EBU,VGluT2,dlpag,990,242,618
3,EBV,VGluT2,vlpag,993,302,525
4,ECZ,VGluT2,dlpag,939,250,525
...,...,...,...,...,...,...
511,WIBC,VGAT,dmpag,921,251,592
512,WIBD,VGAT,dmpag,921,247,580
513,WIBE,VGAT,dmpag,921,241,552
514,WIBF,VGAT,dmpag,993,229,592


In [8]:
# Scale up data
sharptrack_to_brainrender_scale_factor = 10 # Sharp-Track uses a 10um reference atlas so the coordinates need to be scaled to match brainrender's

pag_data["CCF.AllenAP"] *= sharptrack_to_brainrender_scale_factor
pag_data["CCF.AllenDV"] *= sharptrack_to_brainrender_scale_factor
pag_data["CCF.AllenML"] *= sharptrack_to_brainrender_scale_factor

pag_data[["cell.id", "cell.type", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

,cell.id,cell.type,CCF.AllenAP,CCF.AllenDV,CCF.AllenML
0,EBR,VGluT2,9930,2350,5490
1,EBT,VGluT2,9930,2310,5940
2,EBU,VGluT2,9900,2420,6180
3,EBV,VGluT2,9930,3020,5250
4,ECZ,VGluT2,9390,2500,5250
...,...,...,...,...,...
511,WIBC,VGAT,9210,2510,5920
512,WIBD,VGAT,9210,2470,5800
513,WIBE,VGAT,9210,2410,5520
514,WIBF,VGAT,9930,2290,5920


### 3 | Rendering cells with Brainrender
Now that we have the coordinates at the right scale, we can render our cells in Brainrender and colour them according to any metadata we want. We will prepare different renderings in each of the following code chunks.

#### 3.1 | Selecting a subset of cells
We can first take a look at how to use the metadata to select subsets of cells. This will be useful to either render just some of the cells, or to color cells based on some metadata info.

In [18]:
# We can subset cells using any criteria we want. For instance, let's keep cells coming from each hemisphere in separate variables:
cells_hemisphere_right = pag_data.loc[pag_data["PAG.hemisphere"] == "right"]
cells_hemisphere_left = pag_data.loc[pag_data["PAG.hemisphere"] == "left"]
cells_hemisphere_right.head()

,Unnamed: 0,sequencing.id,date.collection,sample.id,sample.number,mouse.id,mouse.sex,mouse.age,mouse.singlehousedays,cell.id,...,RT.machine,RT.time,preamp.date,preamp.machine,preamp.time,date.purification,date.nanodrop,date.qubit,date.bioanalyzer,batch.sequencing_round
1,2,GC.OP.7836.EBT,180518,dmpag_vglut2_180518_c22_EBT,22,CE51.2,F,7.0,0,EBT,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
2,3,GC.OP.7836.EBU,180518,dlpag_vglut2_180518_c23_EBU,23,CE51.2,F,7.0,0,EBU,...,classic,16:09,180603,classic,18:45,180605,180606.0,180606.0,180618.0,1
5,6,GC.OP.7836.EDG,180607,dlpag_vglut2_180607_c13_EDG,13,BJ162.2,M,7.3,0,EDG,...,classic,22:00,180608,non-classic,14:17,180609,180609.0,180609.0,180609.0,1
6,7,GC.OP.7836.EDM,180607,vlpag_vglut2_180607_c19_EDM,19,BJ162.2,M,7.3,0,EDM,...,classic,22:00,180608,non-classic,14:17,180609,180609.0,180609.0,180614.0,1
7,8,GC.OP.7836.EDV,180611,dlpag_vglut2_180611_c4_EDV,4,BJ162.3,M,7.9,4,EDV,...,classic,22:30,180612,non-classic,19:43,180613,180614.0,180614.0,180614.0,1


In [19]:
# We can also use multiple criteria at the same time, such as hemisphere and cell type:
vgat_cells_hemisphere_left = pag_data.loc[(pag_data["PAG.hemisphere"] == "left")&(pag_data["cell.type"] == "VGAT")]
vgat_cells_hemisphere_left.head()

,Unnamed: 0,sequencing.id,date.collection,sample.id,sample.number,mouse.id,mouse.sex,mouse.age,mouse.singlehousedays,cell.id,...,RT.machine,RT.time,preamp.date,preamp.machine,preamp.time,date.purification,date.nanodrop,date.qubit,date.bioanalyzer,batch.sequencing_round
48,49,GC.OP.7836.IAZ,180618,lpag_vgat_180618_c2_IAZ,2,BO73.1,M,8.0,0,IAZ,...,classic,11:15,180623,classic,14:00,180624,180625.0,180625.0,180626.0,1
49,50,GC.OP.7836.IBF,180618,lpag_vgat_180618_c8_IBF,8,BO73.1,M,8.0,0,IBF,...,classic,11:15,180623,classic,14:00,180624,180625.0,180625.0,180626.0,1
50,51,GC.OP.7836.IBS,180618,dmpag_vgat_180618_c21_IBS,21,BO73.1,M,8.0,0,IBS,...,classic,11:15,180623,classic,14:00,180624,180625.0,180625.0,180626.0,1
53,54,GC.OP.7836.IBX,180619,dmpag_vgat_180619_c2_IBX,2,BO73.2,M,8.1,0,IBX,...,non-classic,15:42,180623,non-classic,18:45,180625,180625.0,180625.0,180626.0,1
54,55,GC.OP.7836.IBY,180619,vlpag_vgat_180619_c3_IBY,3,BO73.2,M,8.1,0,IBY,...,non-classic,15:42,180623,non-classic,18:45,180625,180625.0,180625.0,180626.0,1


We can now render a scene adding each subset independently, so we can tweak variables such as color or size separately: 

In [21]:
brainrender.SHOW_AXES = False # Set this back to False

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# // CREATE SCENE //
scene = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params)
# Set add_root = False to focus only on the desired brain area

# // ADD REGIONS AND CELLS//
scene.add_brain_regions(["PAG"], alpha = 0.1, color = "darkgoldenrod", add_labels = False, use_original_color = False, wireframe = False)
scene.add_cells(cells_hemisphere_right, color = "salmon", col_names = column_names, alpha = 1, radius = 25, res = 24)
scene.add_cells(cells_hemisphere_left, color = "skyblue", col_names = column_names, alpha = 1, radius = 25, res = 24)

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # make sure to press 'Esc' to close not 'q' or kernel dies

Added 262 cells to the scene
Added 254 cells to the scene


Rendering scene.
   Press 'q' to Quit


#### 3.2 | Rendering VGAT and VGluT2 cells
We have two ways to do this. We can either follow the example above and assign each cell type to a different variable and render them separately, or we can render all the cells at once and then use the argument `color_by_metadata` to color each cell type differently.

##### 3.2.1 | Render and color separately

In [26]:
brainrender.SHOW_AXES = False # Set this back to False

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# Let's color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
scene = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params)
# Set add_root = False to focus only on the desired brain area

# // ADD REGIONS AND CELLS//
scene.add_brain_regions(["PAG"], alpha = 0.1, color = "darkgoldenrod", add_labels = False, use_original_color = False, wireframe = False)
scene.add_cells(vgat_cells, color = "salmon", col_names = column_names, alpha = 1, radius = 25, res = 24)
scene.add_cells(vglut2_cells, color = "skyblue", col_names = column_names, alpha = 1, radius = 25, res = 24)

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # press 'Esc' to close not 'q' or kernel dies

Added 253 cells to the scene
Added 263 cells to the scene


Rendering scene.
   Press 'q' to Quit


##### 3.2.2 | Render cells together and color by metadata
Here, we render all the cells and color them with the PAG subdivision they were acquired from. We can use this to manually curate the location of each cells after visual inspection. We can further curate the PAG subdivision by following to the next section, where we used the `kim_unified_25um_v0.1` atlas to extract the brain area from the coordinates. We will go over all the possibilities and then pool the results from the different approaches to populate a curated column in the metadata, which will be used for final plotting and further scRNA-seq data analysis.

In [29]:
# // DEFAULT SETTINGS //
# Change some of the default settings
brainrender.SHOW_AXES = False # Set this back to False
brainrender.SHADER_STYLE = "cartoon" # affects the look of rendered brain regions, values can be: ["metallic", "plastic", "shiny", "glossy", "cartoon"] and can be changed in interactive mode
brainrender.BACKGROUND_COLOR = "white" # color of the background window (defaults to "white", try "blackboard")
brainrender.ROOT_COLOR = [0.4, 0.4, 0.4] # color of the overall brain model's actor (defaults to [0.4, 0.4, 0.4])
brainrender.ROOT_ALPHA = 0.2 # transparency of the overall brain model's actor (defaults to 0.2)
brainrender.DEFAULT_SCREENSHOT_NAME = "PAG_scRNAseq_cells" # screenshots will have this name and the time at which they were taken
brainrender.DEFAULT_SCREENSHOT_TYPE = ".png" # png, svg or jpg supported
brainrender.DEFAULT_SCREENSHOT_SCALE = 1 # values >1 yield higher resolution screenshots
brainrender.SCREENSHOT_TRANSPARENT_BACKGROUND = True # whether to save screenshots with transparent background

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# // CREATE SCENE //
scene = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params)
# Set add_root = False to focus only on the desired brain area

# // ADD REGIONS AND CELLS//
scene.add_brain_regions(["PAG"], alpha = 0.1, color = "darkgoldenrod", add_labels = False, use_original_color = False, wireframe = False)
scene.add_cells(pag_data, # pandas dataframe containing x,y,z coordinates
    color = dict(dmpag = "cornflowerblue", dlpag = "darkorange", lpag = "forestgreen", vlpag = "firebrick"), # str, list of colors specifying the color of each cell.
    color_by_region = None, # bool. If true the cells are colored according to the color of the brain region they are in
    color_by_metadata = "PAG.areamanualregistration", # str, if the name of a column of the coords dataframe is passed, cells are colored according to their value for that column. 
    radius = 25, # int, radius of spheres used to render the cells (Default value = 25). Will need adjustment depending on resolution of used atlas.
    res = 24, # int, resolution of spheres used to render the cells (Default value = 3)
    alpha = 1, # float, transparency of spheres used to render the cells (Default value = 1)
    col_names = column_names, # list of strings with names of pandas dataframe columns. Should be a list of 3 columns with the x, y, z coordinates.
    regions = None) # if a list of brain regions acronym is passed, only cells in these regions will be added to the scene

# # // EXPORT AS HTML //
# # Export to a .html file that can be opened in a browser to render an interactive brainrender scene
# scene.export_for_web("output/PAG_scRNAseq_cells.html") # you can pass a filepath where to save the scene


# # // MAKE VIDEO //
# # Create an instance of VideoMaker with our scene
# vm = VideoMaker(scene,
#     save_fld = save_folder, # folder where to save video
#     save_name = "PAG_video_scRNAseq_cells", # video name
#     #video_format = "mp4", # defaults to mp4
#     #duration = 3, # video duration in seconds (defaults to 3)
#     #niters = 60, # number of iterations (frames) when creating the video (defaults to 60)
#     #fps = 30 # framerate of video (defaults to 30)
#     ) 

# # Make a video and specify how the scene rotates at each frame. You can also change several parameters (destination folder, video name, fps, duration, etc.) specified above
# vm.make_video(azimuth = 1, elevation = 1, roll = 0, # rotation in degrees per frame on the relative axis    
#     save_fld = save_folder, save_name = "PAG_video_scRNAseq_cells",
#     video_format = "avi", duration = 5, niters = 50, fps = 30)


# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # press 'Esc' to close not 'q' or kernel dies

Added 516 cells to the scene


Rendering scene.
   Press 'q' to Quit


### 4 | Using the enhanced Franklin-Paxinos atlas (Chon et al. 2019) to obtain PAG subdivisions from CCF coordinates
Another useful functionality of Brainrender is using `atlas.structure_from_coords()` to obtain the PAG subdivision each cell belongs to from the registered coordinates.

#### 4.1 | Using kim_unified_25um_v0.1

In [ ]:
# Scale up data: sharptrack uses a 10um reference atlas, brainrender's default is 1um, and the kim atlas is either 50um or 25um, so the coordinates need to be scaled
brainrender_to_kim_25_scale_factor = 0.04 # the kim atlas we are using is at 25um resolution so the coordinates need to be scaled to match the root

pag_data["CCF.AllenAP"] *= brainrender_to_kim_25_scale_factor
pag_data["CCF.AllenDV"] *= brainrender_to_kim_25_scale_factor
pag_data["CCF.AllenML"] *= brainrender_to_kim_25_scale_factor

pag_data[["cell.id", "cell.type", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

In [ ]:
brainrender.SHOW_AXES = False # Set this back to False

# Create a variable containing the XYZ coordinates of the cells.
column_names = ["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"] # name of the columns containing the CCF coordinates

# To use the atlas with PAG subdivisions, try:
# kim_unified_50um_v0.1
# kim_unified_25um_v0.1

# Let's color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
# Create a scene with no title. You can also use scene.add_text to add other text elsewhere in the scene
scene = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params, atlas = "kim_unified_25um_v0.1")
# Set add_root = False to focus only on the desired brain area

# // ADD CELLS//
scene.add_cells(vgat_cells, color = "salmon", color_by_region = False, col_names = column_names, radius = 1, res = 24, alpha = 1)
scene.add_cells(vglut2_cells, color = "skyblue", color_by_region = False, col_names = column_names, radius = 1, res = 24, alpha = 1)

# // ADD REGIONS //
scene.add_brain_regions(["PAG"],
    colors = get_random_colors(1), use_original_color = False)
# scene.add_brain_regions(["DMPAG", "DLPAG", "LPAG", 
# Try this:AG"],
#     color = dict(DLPAG = "blackboard", DMPAG = "olivedrab", LPAG = s"li[PAG = "red"), "nal_color = 
# // RENDER IVELY //
# Render interactively. You

# Or this: # pag_dict = dict(DLPAG = "blackboard", DMPAG = "olivedrab", LPAG = "lightgreen", VLPAG = "red")
# scene.add_brain_regions(pag_dict.keys(),
#     colors = pag_dict.values(), use_original_color = False)
can press "s" to take a screenshot
scene.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # press 'Esc' to close not 'q' or kernel dies

In [ ]:
pag_data.head()
pag_data["brainrender.area"] = "empty"
pag_data.head()

In [ ]:
cell_region = scene.atlas.structure_from_coords([94.0, 397.2, 219.6], as_acronym = True)
cell_region
#pag_data.head()

In [ ]:
# Try the following to get the PAG area from the coordinates.
# [AP, SI, LR] = [1320, 800, 1140] --> 10um resolution (sharp-track)
# [AP, SI, LR] = [528, 320, 456] --> 25um resolution (kim_unified)
# in scene.atlas.structure_from_coords axis 0 is SI, 1 is AP, and 2 is LR, so [SI, AP, LR]

# cell_region = scene.atlas.structure_from_coords([94.0, 397.2, 219.6], as_acronym = True)
# cell_region

# coordinates = pag_data[["CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]] # Original order
# coordinates = pag_data[["CCF.AllenDV", "CCF.AllenAP", "CCF.AllenML"]]

pag_data["brainrender.area"] = "empty"

for index, row in pag_data.iterrows():
    #print(scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True))
    #print(pag_data[row["cell.id"]])
    #pag_data[row["brainrender.area"]] = scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True)
    cell_region = scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True)
    pag_data.at[index, "brainrender.area"] = cell_region
    #pag_data = pag_data.append({"brainrender.area": str(scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True))}, ignore_index=True)

pag_data.head()
#pag_data.tail()

In [ ]:
you need first to create a list with the data you need, then you can add that as a column of the df
data = []
for index, row in pag_data.iterrows():
    data.append({"brainrender.area": print([scene.atlas.structure_from_coords([row["CCF.AllenDV"], row["CCF.AllenAP"], row["CCF.AllenML"]], as_acronym = True)])}, ignore_index=True)
and then pagadata["brainrender_area"] = data
there might be a more elegant way to do it without the list inbetween, but this is a safe way

#### 4.2 | Using kim_unified_50um_v0.1

In [ ]:
# Scale up data: sharptrack uses a 10um reference atlas, brainrender's default is 1um, and the kim atlas is either 50um or 25um, so the coordinates need to be scaled
brainrender_to_kim_50_scale_factor = 0.02 # the kim atlas we are using is at 50um resolution so the coordinates need to be scaled to match the root

pag_data["CCF.AllenAP"] *= brainrender_to_kim_50_scale_factor
pag_data["CCF.AllenDV"] *= brainrender_to_kim_50_scale_factor
pag_data["CCF.AllenML"] *= brainrender_to_kim_50_scale_factor

pag_data[["cell.id", "cell.type", "CCF.AllenAP", "CCF.AllenDV", "CCF.AllenML"]]

### 5 | Dual scenes \[work in progress\]
We can use `DualScene` or `MultiScene` to render two or more scenes side by side, each containing a different subset of our data for easy comparison.

In [ ]:
# Let's color cells according to whether they are excitatory or inhibitory:
vgat_cells = pag_data.loc[pag_data["cell.type"] == "VGAT"]
vglut2_cells =  pag_data.loc[pag_data["cell.type"] == "VGluT2"]

# // CREATE SCENE //
# Create a scene with no title. You can also use scene.add_text to add other text elsewhere in the scene
scene_vgat = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params)
scene_vglut2 = Scene(display_inset = True, title = None, camera = "sagittal", screenshot_kwargs = screenshot_params)

# // CREATE DUAL SCENE //
scene_dual = DualScene[scene_vgat, scene_vglut2]

# // ADD CELLS//
scene_vgat.add_cells(vgat_cells, color = "salmon", col_names = column_names, res = 24)
scene_vglut2.add_cells(vglut2_cells, color = "skyblue", col_names = column_names, res = 24)

# // RENDER INTERACTIVELY //
# Render interactively. You can press "s" to take a screenshot
scene_dual.render(interactive = True, video = False, camera = "sagittal", zoom = 1) # press 'Esc' to close not 'q' or kernel dies